<a href="https://colab.research.google.com/github/otitamario/sp-pa-gep/blob/main/experiments/Example5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone the repository into Colab runtime
!git clone https://github.com/otitamario/sp-pa-gep.git

# Move into repo root
%cd sp-pa-gep

# Make sure Python sees the project root
import sys
sys.path.append(".")

Cloning into 'sp-pa-gep'...
remote: Enumerating objects: 187, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 187 (delta 79), reused 73 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (187/187), 3.44 MiB | 8.73 MiB/s, done.
Resolving deltas: 100% (79/79), done.
/content/sp-pa-gep


In [ ]:
import os
# =========================
# OUTPUT DIRECTORY (save plots here)
# =========================
FIGDIR = "figures"
os.makedirs(FIGDIR, exist_ok=True)

In [ ]:
#Projection onto box C
import numpy as np
import matplotlib.pyplot as plt
import os


def proj_box(x, low=-2.0, high=5.0):
    return np.clip(x, low, high)

#Proximal mapping P(x)
def prox_quartic(x, maxit=50, tol=1e-12):
    nx = np.linalg.norm(x)
    if nx == 0.0:
        return x.copy()
    a = nx**2  # ||x||^2

    # Solve t + a t^3 = 1 for t in (0,1]
    # good initial guess
    t = 1.0 / (1.0 + a)

    for _ in range(maxit):
        f = t + a * t**3 - 1.0
        df = 1.0 + 3.0 * a * t**2
        t_new = t - f / df
        # keep positive
        if t_new <= 0:
            t_new = 0.5 * t
        if abs(t_new - t) < tol:
            t = t_new
            break
        t = t_new

    return t * x

#Build a PSD matrix A>=0
def make_psd_matrix(m=5, seed=0):
    rng = np.random.default_rng(seed)
    B = rng.uniform(-2, 2, size=(m, m))
    A = B.T @ B  # PSD
    # normalize to keep scale moderate
    A = A / np.linalg.norm(A, ord=2)
    return A

#Define F(x)=Ax + P(x)
def make_F(A):
    def F(x):
        return A @ x + prox_quartic(x)
    return F

#Inner fixed-point solver for the resolvent
#u=PC(x−rF(u))

def resolvent_inner_fixed_point(x, r, F, inner_tol=1e-10, inner_maxit=200):
    z = proj_box(x)  # initial guess inside C
    for it in range(1, inner_maxit + 1):
        z_new = proj_box(x - r * F(z))
        if np.linalg.norm(z_new - z) <= inner_tol:
            return z_new, it
        z = z_new
    return z, inner_maxit

def residual_R(x, r, F):
    return np.linalg.norm(x - proj_box(x - r * F(x)))


In [ ]:
import numpy as np
import src.benchkit as bk

# --- problem data (same as your notebook) ---
m = 5
A = make_psd_matrix(m=m, seed=123)
F = make_F(A)

r = 0.5
N = 150
inner_tol = 1e-10
inner_maxit = 200

x0 = np.ones(m) * 4.0
u_anchor = np.zeros(m)

# --- step functions for benchkit ---
def make_sppa_step():
    def step_fn(x, k):
        alpha = 1.0 / (k + 2.0)
        u_n, it = resolvent_inner_fixed_point(x, r, F, inner_tol=inner_tol, inner_maxit=inner_maxit)
        x_new = alpha * u_anchor + (1.0 - alpha) * u_n
        return x_new, {"inner_iters": it}
    return step_fn

def make_wppa_step():
    def step_fn(x, k):
        alpha = 1.0 / (k + 2.0)
        u_n, it = resolvent_inner_fixed_point(x, r, F, inner_tol=inner_tol, inner_maxit=inner_maxit)
        x_new = alpha * x + (1.0 - alpha) * u_n
        return x_new, {"inner_iters": it}
    return step_fn

def residual_fn(x):
    return residual_R(x, r, F)

# --- run ---
logs_sppa, sum_sppa = bk.run(
    method_name="SPPA",
    x0=x0,
    max_iter=N,
    stop_tol_step=1e-12,   # keep full N for fair curves (or set 1e-6 to early-stop)
    step_fn=make_sppa_step(),
    residual_fn=residual_fn,
    error_fn=None,         # x* unknown in this example
)

logs_wppa, sum_wppa = bk.run(
    method_name="WPPA",
    x0=x0,
    max_iter=N,
    stop_tol_step=1e-12,
    step_fn=make_wppa_step(),
    residual_fn=residual_fn,
    error_fn=None,
)

# --- figures (step + residual; error omitted automatically) ---
bk.make_standard_plots(
    logs_by_method={"SPPA": logs_sppa, "WPPA": logs_wppa},
    outdir="figures",
    tag="ex52_nonlinear",
    plot_residual=True,
    plot_error=False,
)

# --- LaTeX table ---
print(bk.latex_table(
    [sum_sppa, sum_wppa],
    caption="CPU/accuracy summary for Example 5.2 (nonlinear monotone EP).",
    label="tab:ex52",
))

\begin{table}[!ht]
\centering
\begin{tabular}{lrrrrrr}
\hline
Method & Iters & Total (s) & Avg resolvent (s) & Avg residual (s) & Final $\|x_n-\bar x\|$ & Final $R(x_n)$\\
\hline
SPPA & 150 & 0.5490 & 0.00362 & 0.00002 & - & 7.2922e-04 \\
WPPA & 150 & 0.8473 & 0.00559 & 0.00004 & - & 2.2910e-03 \\
\hline
\end{tabular}
\caption{CPU/accuracy summary for Example 5.2 (nonlinear monotone EP).}
\label{tab:ex52}
\end{table}

